In [35]:
import torch
import time
from torch_geometric.utils import to_dense_adj

In [36]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step_sparse(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        size = torch.Size([edge_index.size(1), edge_index.size(1)] + list(edge_weight.size())[1:])
        adj = torch.sparse.IntTensor(edge_index, edge_weight, size = size)
        
        #adj = to_dense_adj(edge_index)[0]
        for k in range(2, x+1):
            adj_k = 1/k * (torch.sparse.mm(adj, adj) - adj)
            
            #adj_k._values = (adj_k._values()>0)*1
            #adj = adj+adj_k

        #print(adj.to_dense())
        return adj
            
    return edge_index

avg_passed = 0
n= 0
for i in range(1000):   
    start = time.time_ns()
    hop2 = one_step_sparse(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Sparse: \n", hop2)
print()
print("Avg. time: \n", avg_passed/n)

Original Edge_index: 
 tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])

Hop2-Sparse: 
 tensor(indices=tensor([[0, 0, 1, 1, 2, 2],
                       [1, 2, 3, 4, 4, 5]]),
       values=tensor([1., 1., 1., 1., 1., 1.]),
       size=(6, 6), nnz=6, layout=torch.sparse_coo)

Avg. time: 
 997365.868263473


In [37]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        adj = to_dense_adj(edge_index)[0]
        for k in range(2,x+1):
            adj_k = (torch.linalg.matrix_power(adj, k)>0)*1 - adj
            
            edge_index_k = (adj_k > 0).nonzero().t()
            edge_weight_k = 1/k * torch.ones(edge_index_k.size(1))
            edge_index = torch.cat((edge_index, edge_index_k), dim=1)
            edge_weight = torch.cat((edge_weight, edge_weight_k), dim=0)

    return edge_index, edge_weight

avg_passed = 0
n= 0
for i in range(1000):   
    start = time.time_ns()
    hop2, w2 = one_step(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Edge-Index:\n", hop2)
print("Edge-Weights:\n", w2)
print()
print("Avg. time:", avg_passed/n)

Original Edge_index: 
 tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])

Hop2-Edge-Index: tensor([[0, 0, 1, 1, 2, 2, 0, 0, 0],
        [1, 2, 3, 4, 4, 5, 3, 4, 5]])
Edge-Weights: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000])

Avg. time: 997855.6390977444
